## 1 - Packages ##

First, you need to import all the packages that you will need during this assignment. 
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [pandas](pandas.pydata.org/) is an important package for Python data analysis.
- [jdc](https://alexhagen.github.io/jdc/) : Jupyter magic that allows defining classes over multiple jupyter notebook cells.

In [1]:
!pip install jdc
import numpy as np
import pandas as pd
import jdc

## 2 - Required Methods ##

### 2.1 - How to display the column names of a given dataset ###
- Pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)
- Pandas [pandas.DataFrame.columns](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html)

In [2]:
# Define a DataFrame object
df_sample = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])

# print the content of the DataFrame
print(df_sample)

   a  b  c
0  1  2  3
1  4  5  6
2  7  8  9


In [3]:
# Print the column names of the dataset
print(df_sample.columns)
print(df_sample.columns[0])

Index(['a', 'b', 'c'], dtype='object')
a


#### Graded Excercise #### 
Print out the column names of the training set that will be used in this assignment

In [4]:
# Will be graded

# Load the training data that we will use in this assignment
df_train = pd.read_csv('train.csv')

# Print the column names of the training data
### START CODE HERE ### (≈ 1 line of code)
# Replace ??? with the correct code
print( df_train.columns )
### END CODE HERE ###

Index(['wesley', 'romulan', 'poetry', 'honor', 'tea', 'barclay', 'class'], dtype='object')


### 2.2 - How to determine the unique values in a given array ###
- [numpy.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html)

In [5]:
# Return the unique values in an array
arr = np.array([1, 1, 1, 1, 2, 2, 2, 3, 3, 4])
np.unique(arr)

array([1, 2, 3, 4])

In [6]:
# Return both the unique values and the count of each value
np.unique(arr, return_counts=True)

(array([1, 2, 3, 4]), array([4, 3, 2, 1]))

In [7]:
# You can store the uniuqe values and their counts in variables
values, counts = np.unique(arr, return_counts=True)
print("Unique values are", values)
print("Corresponding counts are", counts)

Unique values are [1 2 3 4]
Corresponding counts are [4 3 2 1]


#### Graded Excercise #### 
Print out the unique values in the column of "wesley" and their corresponding counts
- hint: the "wesley" column can be obtained by using df['wesley']

In [8]:
### START CODE HERE ### (≈ 1 line of code)
values, counts = np.unique(df_train['wesley'], return_counts=True)
### END CODE HERE ###
print("Unique values in 'wesley' are", values)
print("Corresponding counts are", counts)

Unique values in 'wesley' are [0 1]
Corresponding counts are [408 392]


### 2.3 - DataFrame.where ###
-[DataFrame.where](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.where.html)
- Identify a subset based on certain conditions

In [9]:
# For df_sample above, we want to find out items whose a >= 4
df_sample.where(df_sample['a'] >= 4)

,a,b,c
0,NaN,NaN,NaN
1,4.0,5.0,6.0
2,7.0,8.0,9.0


As you can see, if items are evaluated to be false, they are replaced by "NaN".
- [DataFrame.dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
- It can drop the "NaN" rows

In [10]:
df_sample.where(df_sample['a'] >= 4).dropna()

,a,b,c
1,4.0,5.0,6.0
2,7.0,8.0,9.0


In [11]:
# Drop NaN and return a specific column
df_sample.where(df_sample['a'] >= 4).dropna()['a']

1    4.0
2    7.0
Name: a, dtype: float64

#### Graded Exercise ####
- Return the list of items where tea=1 in our training set
- Only show the "class" column in your result

In [12]:
### START CODE HERE ###
# Replace ??? with the correct code
df_train.where(df_train['tea'] == 1).dropna()['class']
### END CODE HERE ###

2      0.0
7      1.0
9      1.0
10     1.0
15     0.0
      ... 
788    0.0
792    0.0
793    0.0
797    1.0
799    0.0
Name: class, Length: 403, dtype: float64

## 3 - Fundamentals in Decision Tree ##

### 3.1 - Entropy ###
- As usual, we will define a class named "Decision_Tree"
- We will implement entropy function:
$$ H = -\sum_{i=1}^{n} P_{i} * \log_{2}P_{i}$$

In [13]:
class Decision_Tree():
    def entropy(self, column):
        """
        Calculate the entropy of a given data column.
        column: the data column
        """
        
        # the list that will contain every -pi * log(pi)
        ent_list = []
        # print(len(column))
        
        ### START CODE HERE ###
        # Determine the unique values in the column and their corresponding counts of each unique value
        values, counts = np.unique(column, return_counts=True)
        # print(values)
        # print(counts)
        
        # The number of disctint values
        num_distinct_values = len(values)
        # the total number of items in the column
        total_items_in_column = len(column)
        
        for i in range(num_distinct_values):
            # calculate the probability pi for the ith value 
            p_i = counts[i]/np.sum(counts)
            # calculate -pi * log(pi)
            ent_i =  -p_i * np.log2(p_i)
            #put the result into the list
            ent_list.append(ent_i)
        
        ### END CODE HERE ###

        return np.sum(ent_list)

### 3.2 - Information Gain ###
$$ IG(S|a) = entropy(S) - \sum_{v \in values(a)} \frac {|S_{v}|} {|S|} * entropy(S_{v})$$
where
- $IG(S|a)$ means the information gain if we split data S using attribute a
- $|S_{v}|$ means the number of items with $a = v$
- $|S|$ means the total number of items in S
- $\frac {|S_{v}|} {|S|}$ is the probability of $ a = v $

In [14]:
%%add_to Decision_Tree
def Infomation_Gain(self, S, entropy_before_splitting, a, class_name = "class"):
    """
    Calculate the information gain of a dataset. This function takes four parameters:
    1. S: the overall dataset (See the equation above)
    2. entropy_before_splitting: the entropy before splitting
    3. a: the attribute that we will use to split the data (See the equation above)
    4. class_name = the class that the set of data is classified as
    """    
    # the list that will contain the weighted entropy of each child, i.e., Pv * Hv, 
    #       where Pv is the probability of being in the child node v, and
    #       Hv is the entropy of child node v: entropy(Sv).
    H_list = []
    
    ### START CODE HERE ###
    #determine the unique values and their corresponding counts for the split attribute 
    unique_vals, counts= np.unique(S[a], return_counts=True)
    
    #calculate the total number of items in S
    total_S = len(S)
    #Calculate the total number of unnique values with regard to attribute a
    total_Sv = len(unique_vals)

    # print(unique_vals)
    # print(counts)
    # print(S, "\n")
    # print(entropy_before_split)
    # print(a, "\n")
    # print(len(unique_vals))
    # print(len(S))
    
    for i in range(total_Sv):
        # the probablity of being in the ith child node
        P_i = counts[i]/np.sum(counts)
        # the value v of the ith child
        v = unique_vals[i]
        # the subset Sv, where a = v
        # hint: use DataFrame.where and only return the "class" column
        S_v = S.where(S[a] == v).dropna()[class_name]
        # the entropy of child node Sv
        H_i = self.entropy(S_v)
        # put P_i * H_i into the H_list 
        H_list.append(P_i * H_i)
    
    # calculate the conditional entropy based on H_list
    conditional_entropy = np.sum(H_list)
    ### END CODE HERE ###
    
    #Calculate the information gain
    Information_Gain = entropy_before_splitting - conditional_entropy
    return Information_Gain

## 4 - Experiment ##

In [15]:
DT = Decision_Tree()

### START CODE HERE ###
# Data set has been obtained in df_train above
# Calculate the entropy before splitting based on df_train
entropy_before_split = DT.entropy(df_train)
print("Entropy before splitting is ", entropy_before_split)
# 
#calcuate the information gain for attribute "ig"
ig = DT.Infomation_Gain(df_train, entropy_before_split, 'barclay')
print("Information gain for barclay is ", ig)
### END CODE HERE ###

Entropy before splitting is  0.9965700521510767
Information gain for barclay is  0.16029078947930264
